<a href="https://colab.research.google.com/github/louzeiro/desafios/blob/main/smtr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import json
import pandas as pd
import datetime
import time
import pytz
tz = pytz.timezone('America/Sao_Paulo')

url_api = "http://citgisbrj.tacom.srv.br:9977/gtfs-realtime-exporter/findAll/json"
tempo = 60

In [65]:

instante = datetime.datetime.now()
url_api = "http://citgisbrj.tacom.srv.br:9977/gtfs-realtime-exporter/findAll/json"
r = requests.get(url_api)
json_request = json.loads(r.text)

data = pd.DataFrame.from_dict(json_request, orient='columns')
data['coleta_atual'] = pd.to_datetime(instante)

data.head(5)

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.394143,-22.963703,13428,0,47,1660593424,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660606287,2022-08-15 23:31:59.745827
1,-43.361862,-22.915712,47031,0,31,1660601662,41,MADUREIRA X TERMINAL RECREIO (EXPRESSO),ITI VOLTA - 41 MADUREIRA X TERMINAL RECREIO (E...,1660606316,2022-08-15 23:31:59.745827
2,-43.365157,-23.001053,47026,5,29,1660605281,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (E...,1660606289,2022-08-15 23:31:59.745827
3,-43.335785,-22.869882,47011,7,9,1660592457,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660606293,2022-08-15 23:31:59.745827
4,-43.366600,-23.001145,86811,3,0,1660602561,25A,MATO ALTO X ALVORADA (PARADOR),ITI IDA - 25A MATO ALTO X ALVORADA (PARADOR),1660606303,2022-08-15 23:31:59.745827


In [19]:
instante_atual

datetime.datetime(2022, 8, 15, 22, 48, 14, 67667)

In [132]:
tempo = 60
print('Aguardando {} minutos.'.format(tempo/60))
time.sleep(tempo) 

#data['coleta_antiga'] = data['coleta_atual']

instante = datetime.datetime.now()

r = requests.get(url_api)
json_request = json.loads(r.text)

data2 = pd.DataFrame.from_dict(json_request, orient='columns')
data2['coleta_atual'] = pd.to_datetime(instante)
dim = data.shape
new_vei = 0
for idx in data2.index:

  # se o veiculo não está no dataframe, então add
  if not data2['vei_nro_gestor'][idx] in data.vei_nro_gestor.values: #verificando se o veiculo ja esta na BD
    #print('O veículo {} já esta está na BD'.format(data2['vei_nro_gestor'][idx]))
    #print(idx)
    #print('---------Add novo veiculo ao df----------------------------------------------------')
    novo_veiculo = data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    data = data.append(novo_veiculo, ignore_index=True)
    new_vei = new_vei+1
  
  #veículo já está no df
  
  else:
    veiculo = data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values

    #verificando se as coordenadas são diferentes
    if (veiculo.longitude.values != data2.loc[idx].longitude or 
        veiculo.latitude.values != data2.loc[idx].latitude):
      # então atualiza o veiculo no df
      #print(idx)
      #print('--------- Atualizando a posição do veículo {}--------------------------'.format(data2['vei_nro_gestor'][idx]))
      data.loc[idx_vei] = data2.loc[idx].values
    
    #as coordenadas são as mesma, verificando se a mais de 1 min
    else:
      tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual
      if (tempo.dt.seconds > 60).all():
        print('----------------------------------')
        print('O veiculo {} foi atualizado a {} segundos, logo seus dados não serão atualizados na base'.format(data2['vei_nro_gestor'][idx], tempo))
        print(veiculo.values)
        print('----------------------------------')
        print(data2.loc[idx].values)
        print()
        print('Index do vei no data ===> {}'.format(idx_vei))
        print(data.loc[idx_vei].values)
        print(data.shape)
        data.drop(idx_vei, axis=0, inplace=True)
        print(data.shape)
#        data.drop([idx_vei], inplace=True)

        #data.drop([idx_vei], axis=0, inplace=True)
        #data.loc[idx_vei] = data2.loc[idx].values


Aguardando 1.0 minutos.
----------------------------------
O veiculo 47002 foi atualizado a 65   0 days 00:02:07.100151
Name: coleta_atual, dtype: timedelta64[ns] segundos, logo seus dados não serão atualizados na base
[[-43.366377 -23.000713 '47002' 2 0 '1660605726' '53A'
  'TERMINAL SULACAP X ALVORADA (EXPRESSO)'
  'ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (EXPRESSO)' '1660615075'
  Timestamp('2022-08-16 01:58:11.149116')]]
----------------------------------
[-43.366377 -23.000713 '47002' 4 0 '1660605726' '53A'
 'TERMINAL SULACAP X ALVORADA (EXPRESSO)'
 'ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (EXPRESSO)' '1660615197'
 Timestamp('2022-08-16 02:00:18.249267')]

Index do vei no data ===> [65]
[[-43.366377 -23.000713 '47002' 2 0 '1660605726' '53A'
  'TERMINAL SULACAP X ALVORADA (EXPRESSO)'
  'ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (EXPRESSO)' '1660615075'
  Timestamp('2022-08-16 01:58:11.149116')]]
(66, 11)
(65, 11)
----------------------------------
O veiculo 86840 foi atua

# Completo

In [ ]:
instante = datetime.datetime.now(tz=tz)
r = requests.get(url_api)
json_request = json.loads(r.text)

data = pd.DataFrame.from_dict(json_request, orient='columns')
data['coleta_atual'] = instante

for i in range(50):
  print('Aguardando {} minutos.'.format(tempo/60))
  time.sleep(tempo) 

  #data['coleta_antiga'] = data['coleta_atual']

  instante = datetime.datetime.now(tz=tz)


  r = requests.get(url_api)
  json_request = json.loads(r.text)

  data2 = pd.DataFrame.from_dict(json_request, orient='columns')
  data2['coleta_atual'] = instante
  dim = data.shape
  new_vei = 0
  for idx in data2.index:

    # se o veiculo não está no dataframe, então add
    if not data2['vei_nro_gestor'][idx] in data.vei_nro_gestor.values: #verificando se o veiculo ja esta na BD
      #print('O veículo {} já esta está na BD'.format(data2['vei_nro_gestor'][idx]))
      #print(idx)
      #print('---------Add novo veiculo ao df----------------------------------------------------')
      novo_veiculo = data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
      data = data.append(novo_veiculo, ignore_index=True)
      new_vei = new_vei+1

    #veículo já está no df

    else:
      veiculo = data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
      idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values

      #verificando se as coordenadas são diferentes
      if (veiculo.longitude.values != data2.loc[idx].longitude or 
          veiculo.latitude.values != data2.loc[idx].latitude):
        # então atualiza o veiculo no df
        #print(idx)
        #print('--------- Atualizando a posição do veículo {}--------------------------'.format(data2['vei_nro_gestor'][idx]))
        data.loc[idx_vei] = data2.loc[idx].values

      #as coordenadas são as mesma, verificando se a mais de 1 min
      else:
        dif_tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual
        if (dif_tempo.dt.seconds > 60).all():
          print('----------------------------------')
          print('O veiculo {} foi atualizado a {} segundos, logo seus dados não serão atualizados na base'.format(data2['vei_nro_gestor'][idx], dif_tempo))
          #print(veiculo.values)
          #print('----------------------------------')
          #print(data2.loc[idx].values)
          print()
          #print('Index do vei no data ===> {}'.format(idx_vei))
          #print(data.loc[idx_vei].values)
          #print(data.shape)
          data.drop(idx_vei, axis=0, inplace=True)
          #print(data.shape)
  #        data.drop([idx_vei], inplace=True)

          #data.drop([idx_vei], axis=0, inplace=True)
          #data.loc[idx_vei] = data2.loc[idx].values

data.to_csv('infos.csv', index=False)

Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
----------------------------------
O veiculo 90005 foi atualizado a 1   0 days 00:02:01.270397
Name: coleta_atual, dtype: timedelta64[ns] segundos, logo seus dados não serão atualizados na base

Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.
Aguardando 1.0 minutos.


In [ ]:
instante2=datetime.datetime.now(tz=tz)
time.sleep(6)
instante3=datetime.datetime.now(tz=tz)
print(instante2)
print(instante3)
print(instante2-instante3)
instante3.timestamp() - instante2.timestamp()

#outra maneira
data['teste'] = instante2
data['teste2'] = instante3
a = data['teste2'][0] - data['teste'][0]
print(a.total_seconds())
a

In [217]:
print(data2.loc[idx].coleta_atual)
print(veiculo.coleta_atual)

2022-08-16 00:50:22.601119-03:00
8   2022-08-16 00:49:21.813974-03:00
Name: coleta_atual, dtype: datetime64[ns, America/Sao_Paulo]


In [218]:
 dif_tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual
 print(dif_tempo)

8   0 days 00:01:00.787145
Name: coleta_atual, dtype: timedelta64[ns]


# Prefect

In [ ]:
!pip3 install prefect


In [8]:
import prefect

In [10]:
prefect.Flow

prefect.flows.Flow

In [144]:
#data['coleta_atual'] =
pd.to_datetime(data['coleta_atual'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo') - instante

TypeError: ignored

In [142]:
pd.to_datetime(instante).dt.tz_convert('America/Sao_Paulo')

AttributeError: ignored

In [138]:
pd.to_datetime(instante)

Timestamp('2022-08-16 02:46:59.105065')

In [111]:
data.drop([22], axis=0, inplace=True)

In [106]:
dim = data.shape
dim

(66, 11)

In [103]:
data.loc[22].values

array([-43.391992, -22.964722, '13412', 6, 0, '1660594978', '12',
       'PINGO DAGUA X ALVORADA (EXPRESSO)', 'Iti Default', '1660604228',
       Timestamp('2022-08-15 23:33:19.033218')], dtype=object)

In [98]:
data.drop([0], axis=0, inplace=True)
data.head()

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
1,-43.372608,-22.941678,47031,3,44,1660601662,41,MADUREIRA X TERMINAL RECREIO (EXPRESSO),ITI VOLTA - 41 MADUREIRA X TERMINAL RECREIO (E...,1660608449,2022-08-16 00:07:53.258370
2,-43.400770,-22.886433,47026,4,32,1660607894,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI IDA - 53A TERMINAL SULACAP X ALVORADA (EXP...,1660608443,2022-08-16 00:07:53.258370
3,-43.334903,-22.867047,47011,4,35,1660592457,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660608425,2022-08-16 00:07:53.258370
4,-43.466297,-23.014148,86811,5,45,1660602561,25A,MATO ALTO X ALVORADA (PARADOR),ITI IDA - 25A MATO ALTO X ALVORADA (PARADOR),1660608439,2022-08-16 00:07:53.258370
5,-43.312563,-23.006857,13420,2,0,1660599901,18,RECREIO X JARDIM OCEÂNICO (EXPRESSO),ITI IDA - 18 RECREIO X JARDIM OCEÂNICO (EXPRESSO),1660606497,2022-08-15 23:35:49.830301


In [90]:
data2.loc[idx].values

array([-43.251152, -22.854453, '27205', 6, 0, '1660595123', '42A',
       'GALEAO X MADUREIRA (PARADOR)',
       'ITI VOLTA - 42A GALEAO X MADUREIRA (PARADOR)', '1660607932',
       Timestamp('2022-08-15 23:59:13.456251')], dtype=object)

In [87]:
data.loc[data['vei_nro_gestor']==13412]

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual


dtype('<m8[ns]')

42    True
Name: coleta_atual, dtype: bool

In [71]:
data

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.389973,-22.949527,13428,7,50,1660593424,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660606500,2022-08-15 23:35:49.830301
1,-43.354070,-22.900207,47031,1,17,1660601662,41,MADUREIRA X TERMINAL RECREIO (EXPRESSO),ITI VOLTA - 41 MADUREIRA X TERMINAL RECREIO (E...,1660606506,2022-08-15 23:35:49.830301
2,-43.365592,-22.982485,47026,7,43,1660605281,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (E...,1660606502,2022-08-15 23:35:49.830301
3,-43.326530,-22.856020,47011,1,48,1660592457,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660606506,2022-08-15 23:35:49.830301
4,-43.366652,-23.001065,86811,5,0,1660602561,25A,MATO ALTO X ALVORADA (PARADOR),ITI IDA - 25A MATO ALTO X ALVORADA (PARADOR),1660606517,2022-08-15 23:35:49.830301
...,...,...,...,...,...,...,...,...,...,...,...
60,-43.393693,-22.946480,47009,3,57,1660597204,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI IDA - 53A TERMINAL SULACAP X ALVORADA (EXP...,1660606505,2022-08-15 23:35:49.830301
61,-43.401813,-23.000192,47007,2,47,1660603345,18,RECREIO X JARDIM OCEÂNICO (EXPRESSO),ITI VOLTA - 18 RECREIO X JARDIM OCEÂNICO (EXP...,1660606516,2022-08-15 23:35:49.830301
62,-43.334342,-22.865473,12707,5,0,1660604835,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660606489,2022-08-15 23:35:49.830301
63,-43.430478,-23.005247,47010,1,23,1660603253,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI IDA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660606502,2022-08-15 23:35:49.830301


In [70]:
data2

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.671678,-22.932197,47032,6,57,1660602999,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI IDA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660606499,2022-08-15 23:35:49.830301
1,-43.364868,-23.000672,86887,6,0,1660600844,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660606498,2022-08-15 23:35:49.830301
2,-43.385152,-22.973215,13408,5,14,1660605953,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660606495,2022-08-15 23:35:49.830301
3,-43.312663,-23.006803,44731,4,0,1660605369,22,TERMINAL ALVORADA X TERMINAL JARDIM OCEÂNICO (...,ITI IDA - 22 TERMINAL ALVORADA X TERMINAL JARD...,1660606500,2022-08-15 23:35:49.830301
4,-43.459302,-23.012717,90002,5,3,1660604337,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI VOLTA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660606520,2022-08-15 23:35:49.830301
...,...,...,...,...,...,...,...,...,...,...,...
58,-43.359267,-22.909378,13418,4,30,1660592156,46,PENHA X ALVORADA (EXPRESSO),ITI VOLTA - 46 PENHA X ALVORADA (EXPRESSO),1660606519,2022-08-15 23:35:49.830301
59,-43.389973,-22.949527,13428,7,50,1660593424,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660606500,2022-08-15 23:35:49.830301
60,-43.394692,-23.000228,30525,6,57,1660601058,25A,MATO ALTO X ALVORADA (PARADOR),ITI IDA - 25A MATO ALTO X ALVORADA (PARADOR),1660606492,2022-08-15 23:35:49.830301
61,-43.387562,-22.959593,47005,5,36,1660598718,35A,MADUREIRA X JARDIM OCEANICO (PARADOR),ITI VOLTA - 35A MADUREIRA X JARDIM OCEANICO (P...,1660606492,2022-08-15 23:35:49.830301


In [ ]:
pd.to_datetime(instante)

In [64]:
data['coleta_atual'] - instante

0             0 days 00:00:00
1             0 days 00:00:00
2             0 days 00:00:00
3             0 days 00:00:00
4             0 days 00:00:00
               ...           
64            0 days 00:00:00
65            0 days 00:00:00
66   -1 days +23:54:38.651073
67            0 days 00:00:00
68            0 days 00:00:00
Name: coleta_atual, Length: 69, dtype: timedelta64[ns]

In [58]:
data.coleta_atual

0    2022-08-15 23:15:06.569368
1    2022-08-15 23:15:06.569368
2    2022-08-15 23:15:06.569368
3    2022-08-15 23:15:06.569368
4    2022-08-15 23:15:06.569368
                ...            
64   2022-08-15 23:15:06.569368
65   2022-08-15 23:15:06.569368
66   2022-08-15 23:09:45.220441
67   2022-08-15 23:15:06.569368
68   2022-08-15 23:15:06.569368
Name: coleta_atual, Length: 69, dtype: datetime64[ns]

In [60]:
a = datetime.datetime.now()
time.sleep(6)
b=datetime.datetime.now()
b - a

datetime.timedelta(seconds=6, microseconds=7977)

In [63]:
data.coleta_atual)

pandas.core.series.Series

In [61]:
data.coleta_atual - instante

0             0 days 00:00:00
1             0 days 00:00:00
2             0 days 00:00:00
3             0 days 00:00:00
4             0 days 00:00:00
               ...           
64            0 days 00:00:00
65            0 days 00:00:00
66   -1 days +23:54:38.651073
67            0 days 00:00:00
68            0 days 00:00:00
Name: coleta_atual, Length: 69, dtype: timedelta64[ns]

In [ ]:
tempo = 60
print('Aguardando {} minutos.'.format(tempo/60))
time.sleep(tempo) 

#instante_atual = datetime.datetime.now()
instante_atual = datetime.datetime.now()
#instante_atual_fuso = instante_atual.astimezone(fuso_horario)

r = requests.get(url_api)
json_request = json.loads(r.text)

data2 = pd.DataFrame.from_dict(json_request, orient='columns')
data2['coleta_atual'] = instante_atual



for idx in data2.index:

  # se o veiculo não está no dataframe, então add
  if not data2['vei_nro_gestor'][idx] in data.vei_nro_gestor.values: #verificando se o veiculo ja esta na BD
    #print('O veículo {} já esta está na BD'.format(data2['vei_nro_gestor'][idx]))
    print(idx)
    print('---------Add novo veiculo ao df----------------------------------------------------')
    novo_veiculo = data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    data = data.append(novo_veiculo, ignore_index=True)
  
  else:
    veiculo = data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values

    tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual

    #verificando se as coordenadas são diferentes
    if (veiculo.longitude.values != data2.loc[idx].longitude or 
        veiculo.latitude.values != data2.loc[idx].latitude):
      # então atualiza o veiculo no df
      #print(idx)
      #print('--------- Atualizando a posição do veículo {}--------------------------'.format(data2['vei_nro_gestor'][idx]))
      data.loc[idx_vei] = data2.loc[idx].values


data['intervalo_coleta'] = data['coleta_atual'] - instante_atual
data.loc[data['intervalo_coleta'].dt.seconds > 60]



In [ ]:
#data = data.drop(['momento','intervalo_coleta'], axis=1)

In [37]:
data.loc[data['intervalo_coleta'].dt.seconds > 60]


KeyError: ignored

In [35]:
instante_atual

datetime.datetime(2022, 8, 15, 22, 57, 58, 206180)

In [30]:
data2.loc[idx].values

array([-43.312568, -23.006982, '13404', 1, 5, '1660597956', '50',
       'TERMINAL OLÍMPICO X TERMINAL JARDIM OCEÂNICO (PARADOR)',
       'ITI IDA - 50 TERMINAL OLÍMPICO X TERMINAL JARDIM OCEÂNICO (PARADOR)',
       '1660604082', Timestamp('2022-08-15 22:55:05.545363')],
      dtype=object)

In [33]:

data.columns

Index(['longitude', 'latitude', 'vei_nro_gestor', 'direcao', 'velocidade',
       'inicio_viagem', 'linha', 'nomeLinha', 'nomeItinerario', 'comunicacao',
       'coleta_atual'],
      dtype='object')

In [ ]:

data

In [22]:
data.loc[65].coleta_atual

Timestamp('2022-08-15 22:48:14.067667')

In [24]:
instante_atual

datetime.datetime(2022, 8, 15, 22, 49, 53, 33340)

In [13]:
 data.loc[idx_vei].values

array([[-43.3534, -22.899365, '47005', 0, 45, '1660598718', '35A',
        'MADUREIRA X JARDIM OCEANICO (PARADOR)',
        'ITI VOLTA - 35A MADUREIRA X JARDIM OCEANICO (PARADOR)',
        '1660603106', Timestamp('2022-08-15 22:38:36.860254'),
        Timedelta('0 days 00:00:00')]], dtype=object)

In [12]:
data2.loc[idx].values

array([-43.35212, -22.897838, '47005', 1, 35, '1660598718', '35A',
       'MADUREIRA X JARDIM OCEANICO (PARADOR)',
       'ITI VOLTA - 35A MADUREIRA X JARDIM OCEANICO (PARADOR)',
       '1660603241', Timestamp('2022-08-15 22:41:13.225349')],
      dtype=object)

In [143]:
data.loc[25].coleta_atual

Timestamp('2022-08-15 22:30:50.889801')

In [145]:
instante_atual

datetime.datetime(2022, 8, 15, 22, 31, 54, 316390)

In [137]:
data.loc[3].coleta_atual

Timestamp('2022-08-15 22:28:34.820505')

In [8]:
data.loc[data.intervalo_coleta.dt.seconds > 60]

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual,intervalo_coleta
22,-43.382897,-22.973343,47011,5,56,1660592457,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660603032,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
34,-43.365938,-23.000687,47009,2,0,1660597204,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI IDA - 53A TERMINAL SULACAP X ALVORADA (EXP...,1660597271,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
37,-43.391585,-22.965162,13412,3,0,1660594978,12,PINGO DAGUA X ALVORADA (EXPRESSO),Iti Default,1660602315,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
38,-43.367103,-23.001365,27202,1,15,1660599528,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660603015,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
41,-43.313240,-23.006745,13424,7,0,1660588676,18,RECREIO X JARDIM OCEÂNICO (EXPRESSO),ITI VOLTA - 18 RECREIO X JARDIM OCEÂNICO (EXP...,1660603021,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
42,-43.617652,-22.968927,86859,6,33,1660598433,13,MATO ALTO X ALVORADA (EXPRESSO),Iti Default,1660602944,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150
60,-43.400977,-22.910977,47026,0,44,1660596687,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (E...,1660603026,2022-08-15 22:37:20.156404,-1 days +23:58:43.296150


In [120]:
data.loc[idx_vei].values

array([[-43.382682, -22.973155, '13412', 4, 0, '1660594978', '12',
        'PINGO DAGUA X ALVORADA (EXPRESSO)', 'Iti Default', '1660601554',
        Timestamp('2022-08-15 19:12:55.499915-0300', tz='UTC-03:00')]],
      dtype=object)

In [103]:
data2

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.365797,-23.000640,30518,1,18,1660596673,25A,MATO ALTO X ALVORADA (PARADOR),ITI IDA - 25A MATO ALTO X ALVORADA (PARADOR),1660601243,2022-08-15 19:07:27.667847-03:00
1,-43.346865,-23.000137,12738,1,24,1660597356,35A,MADUREIRA X JARDIM OCEANICO (PARADOR),ITI VOLTA - 35A MADUREIRA X JARDIM OCEANICO (P...,1660601232,2022-08-15 19:07:27.667847-03:00
2,-43.400802,-22.911210,13428,0,41,1660593424,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660601219,2022-08-15 19:07:27.667847-03:00
3,-43.323523,-23.003563,47027,6,37,1660600869,22,TERMINAL ALVORADA X TERMINAL JARDIM OCEÂNICO (...,ITI VOLTA - 22 TERMINAL ALVORADA X TERMINAL J...,1660601233,2022-08-15 19:07:27.667847-03:00
4,-43.365733,-22.986398,12707,0,59,1660600759,46,PENHA X ALVORADA (EXPRESSO),ITI VOLTA - 46 PENHA X ALVORADA (EXPRESSO),1660601239,2022-08-15 19:07:27.667847-03:00
...,...,...,...,...,...,...,...,...,...,...,...
69,-43.365997,-22.973423,44730,5,10,1660600731,40A,ALVORADA X MADUREIRA (EXPRESSO),ITI VOLTA - 40A ALVORADA X MADUREIRA (EXPRESSO),1660601221,2022-08-15 19:07:27.667847-03:00
70,-43.540828,-23.008822,44731,6,27,1660598718,13,MATO ALTO X ALVORADA (EXPRESSO),ITI VOLTA - 13 MATO ALTO X ALVORADA (EXPRESSO),1660601228,2022-08-15 19:07:27.667847-03:00
71,-43.367207,-23.001395,47026,2,0,1660596687,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI VOLTA - 53A TERMINAL SULACAP X ALVORADA (E...,1660601168,2022-08-15 19:07:27.667847-03:00
72,-43.348660,-23.000100,90003,1,33,1660596165,50,TERMINAL OLÍMPICO X TERMINAL JARDIM OCEÂNICO (...,ITI IDA - 50 TERMINAL OLÍMPICO X TERMINAL JARD...,1660601217,2022-08-15 19:07:27.667847-03:00


In [89]:
data.loc[idx]

longitude                                               -43.477295
latitude                                                -23.016963
vei_nro_gestor                                               27212
direcao                                                          4
velocidade                                                       0
inicio_viagem                                           1660588709
linha                                                          25A
nomeLinha                           MATO ALTO X ALVORADA (PARADOR)
nomeItinerario      ITI VOLTA - 25A MATO ALTO X ALVORADA (PARADOR)
comunicacao                                             1660600615
coleta_atual                      2022-08-15 18:54:23.524266-03:00
intervalo_coleta                                   0 days 00:00:00
Name: 1, dtype: object

In [71]:
a = a.drop(a[a.dt.seconds > 60].index)
a.shape

(72,)

In [68]:
a.loc[a.dt.seconds > 60]

13

In [58]:
type(a)

pandas.core.series.Series

In [59]:
a.to_list

<bound method IndexOpsMixin.tolist of 0             0 days 00:00:00
1             0 days 00:00:00
2             0 days 00:00:00
3             0 days 00:00:00
4             0 days 00:00:00
               ...           
80            0 days 00:00:00
81   -1 days +23:46:27.767779
82            0 days 00:00:00
83            0 days 00:00:00
84            0 days 00:00:00
Name: coleta_atual, Length: 85, dtype: timedelta64[ns]>

In [63]:
a[0].dt.seconds>60

AttributeError: ignored

In [ ]:
data_bkp = data
data_bkp

In [49]:
for idx in data2.index:

  # se o veiculo não está no dataframe, então add
  if not data2['vei_nro_gestor'][idx] in data.vei_nro_gestor.values: #verificando se o veiculo ja esta na BD
    #print('O veículo {} já esta está na BD'.format(data2['vei_nro_gestor'][idx]))
    print(idx)
    print('---------Add novo veiculo ao df----------------------------------------------------')
    novo_veiculo = data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    data = data.append(novo_veiculo, ignore_index=True)
  
  else:
    veiculo = data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
    idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values

    tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual

    #verificando se as coordenadas são diferentes
    if (veiculo.longitude.values != data2.loc[idx].longitude or 
        veiculo.latitude.values != data2.loc[idx].latitude):
      # então atualiza o veiculo no df
      print(idx)
      print('--------- Atualizando a posição do veículo {}--------------------------'.format(data2['vei_nro_gestor'][idx]))
      data.loc[idx_vei] = data2.loc[idx].values
  

#data

0
--------- Atualizando a posição do veículo 86886--------------------------
1
--------- Atualizando a posição do veículo 44709--------------------------
2
--------- Atualizando a posição do veículo 90007--------------------------
3
--------- Atualizando a posição do veículo 86874--------------------------
4
--------- Atualizando a posição do veículo 30524--------------------------
5
--------- Atualizando a posição do veículo 13416--------------------------
6
--------- Atualizando a posição do veículo 90005--------------------------
7
--------- Atualizando a posição do veículo 30529--------------------------
8
--------- Atualizando a posição do veículo 12707--------------------------
9
--------- Atualizando a posição do veículo 30520--------------------------
10
--------- Atualizando a posição do veículo 13410--------------------------
11
--------- Atualizando a posição do veículo 12705--------------------------
12
--------- Atualizando a posição do veículo 47026-----------------------

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.416387,-22.988763,13405,1,41,1660590620,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660597903,2022-08-15 21:12:09.364247
1,-43.240258,-22.835823,27205,0,43,1660595123,42A,GALEAO X MADUREIRA (PARADOR),ITI VOLTA - 42A GALEAO X MADUREIRA (PARADOR),1660597912,2022-08-15 21:12:09.364247
2,-43.407985,-22.865605,47025,1,58,1660595045,51,VILA MILITAR X TERMINAL RECREIO (PARADOR),ITI VOLTA - 51 VILA MILITAR X TERMINAL RECREIO...,1660597903,2022-08-15 21:12:09.364247
3,-43.399583,-22.883863,30511,1,0,1660585955,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI IDA - 53A TERMINAL SULACAP X ALVORADA (EXP...,1660597912,2022-08-15 21:12:09.364247
4,-43.418475,-23.001420,13416,1,12,1660597530,18,RECREIO X JARDIM OCEÂNICO (EXPRESSO),Iti Default,1660597898,2022-08-15 21:12:09.364247
...,...,...,...,...,...,...,...,...,...,...,...
78,-43.583542,-22.999412,86804,2,39,1660597739,20,SANTA CRUZ X SALVADOR ALLENDE (EXPRESSO),Iti Default,1660597894,2022-08-15 21:12:09.364247
79,-43.365938,-23.000687,47009,2,0,1660597204,53A,TERMINAL SULACAP X ALVORADA (EXPRESSO),ITI IDA - 53A TERMINAL SULACAP X ALVORADA (EXP...,1660597271,2022-08-15 21:09:22.252315
80,-43.388435,-22.959882,27204,0,0,1660597173,41,MADUREIRA X TERMINAL RECREIO (EXPRESSO),ITI IDA - 41 MADUREIRA X TERMINAL RECREIO (EXP...,1660597913,2022-08-15 21:12:09.364247
81,-43.365623,-23.001173,86850,5,0,1660596942,22,TERMINAL ALVORADA X TERMINAL JARDIM OCEÂNICO (...,ITI VOLTA - 22 TERMINAL ALVORADA X TERMINAL J...,1660597750,2022-08-15 21:09:22.252315


In [7]:
data

(68, 11)

In [ ]:
data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx], data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]]

ValueError: ignored

In [ ]:
data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
44,-43.345247,-22.887632,47023,4,0,1660504407,35,MADUREIRA X ALVORADA (PARADOR),ITI IDA - 35 MADUREIRA X ALVORADA (PARADOR),1660506399,2022-08-14 19:46:51.019601


In [ ]:
data.loc[13] = data2.loc[44]

In [ ]:
#data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]] = 
data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
44,-43.345247,-22.887632,47023,4,0,1660504407,35,MADUREIRA X ALVORADA (PARADOR),ITI IDA - 35 MADUREIRA X ALVORADA (PARADOR),1660506399,2022-08-14 19:46:51.019601


In [ ]:
data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]] = data2.loc[data2['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]

In [10]:
## selecionando o veiculo no df1 com o mesmo vei_nro do idx
veiculo = data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]]
idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values

tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual

#verificando se as coordenadas são diferentes
if (veiculo.longitude.values != data2.loc[idx].longitude or 
    veiculo.latitude.values != data2.loc[idx].latitude):
  # então atualiza o veiculo no df
  data.loc[idx_vei] = data2.loc[idx].values
#else:
  # verifica se a última atualização ocorreu a menos de 1 min
  # estranho isso, pq sempre o tempo entre as consultas é 1 min
 # if tempo.dt.seconds < 60:
  


a


In [38]:
tempo.dt.seconds < 0

31    False
Name: coleta_atual, dtype: bool

In [34]:
idx_vei = data[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].index.values
data.loc[idx_vei]
#data.loc[idx_vei]['longitude']

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
31,-43.365242,-23.000717,30518,1,0,1660592067,25A,MATO ALTO X ALVORADA (PARADOR),ITI VOLTA - 25A MATO ALTO X ALVORADA (PARADOR),1660593270,2022-08-15 19:54:37.395234


In [35]:
data2.loc[idx].values

array([-43.365342, -23.000977, '30518', 6, 21, '1660592067', '25A',
       'MATO ALTO X ALVORADA (PARADOR)',
       'ITI VOLTA - 25A MATO ALTO X ALVORADA (PARADOR)', '1660593392',
       Timestamp('2022-08-15 19:56:45.090080')], dtype=object)

In [36]:
data.loc[idx_vei] = data2.loc[idx].values
data.loc[idx_vei]

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
31,-43.365342,-23.000977,30518,6,21,1660592067,25A,MATO ALTO X ALVORADA (PARADOR),ITI VOLTA - 25A MATO ALTO X ALVORADA (PARADOR),1660593392,2022-08-15 19:56:45.090080


In [13]:
data2['longitude'][idx]

-43.365342

In [14]:
veiculo.longitude =  data2['longitude'][idx]
veiculo.longitude

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


31   -43.365342
Name: longitude, dtype: float64

In [17]:
veiculo.latitude.values 

array([-23.000717])

In [16]:
data2.loc[idx].latitude

-23.000977

In [19]:
veiculo['latitude'] = data2.loc[idx].latitude
veiculo['latitude']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


31   -23.000977
Name: latitude, dtype: float64

In [9]:
veiculo.longitude.values != data2.loc[idx].longitude 

array([ True])

In [ ]:
data2.loc[idx].coleta_atual

Timestamp('2022-08-14 19:46:51.019601')

In [ ]:
veiculo.coleta_atual.values

array(['2022-08-14T18:50:08.921896000'], dtype='datetime64[ns]')

In [ ]:
tempo = data2.loc[idx].coleta_atual - veiculo.coleta_atual
tempo
#tempo.strftime("%M")

13   0 days 00:56:42.097705
Name: coleta_atual, dtype: timedelta64[ns]

In [ ]:
tempo.dt.seconds


13    3402
Name: coleta_atual, dtype: int64

In [ ]:
56*60 + 42

3402

In [ ]:
a =  datetime.datetime.now()
time.sleep(5)
b = datetime.datetime.now()

c = b - a
c

datetime.timedelta(seconds=5, microseconds=4387)

In [ ]:
tempo.dt.seconds > 60

13    True
Name: coleta_atual, dtype: bool

In [ ]:
data.loc[data['vei_nro_gestor']==data2['vei_nro_gestor'][idx]].longitude

13   -43.343817
Name: longitude, dtype: float64

In [ ]:
data.columns

Index(['longitude', 'latitude', 'vei_nro_gestor', 'direcao', 'velocidade',
       'inicio_viagem', 'linha', 'nomeLinha', 'nomeItinerario', 'comunicacao',
       'coleta_atual'],
      dtype='object')

In [ ]:
data.longitude.values

array([-43.365887, -43.247557, -43.313948, -43.345645, -43.34486 ,
       -43.394555, -43.365567, -43.339523, -43.562202, -43.684503,
       -43.481783, -43.378528, -43.467015, -43.343817, -43.440752,
       -43.399787, -43.37242 , -43.647482, -43.569093, -43.323233,
       -43.5117  , -43.465768, -43.300848, -43.365688, -43.336402,
       -43.239517, -43.39421 , -43.246302, -43.433067, -43.367732,
       -43.346322, -43.399438, -43.655107, -43.390847, -43.41212 ,
       -43.436033, -43.64351 , -43.621822, -43.365683, -43.373513,
       -43.281777, -43.336362, -43.276623, -43.312358, -43.312032,
       -43.401747, -43.365658, -43.360785, -43.365915, -43.251683,
       -43.320407, -43.366515, -43.638552, -43.318705, -43.402357])

In [ ]:
print(type(data2.loc[data2.vei_nro_gestor==data2.vei_nro_gestor[32]]))
new_vehicle = data2.loc[data2.vei_nro_gestor==data2.vei_nro_gestor[32]]
data.append(new_vehicle, ignore_index=True)

<class 'pandas.core.frame.DataFrame'>


,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.365887,-23.000707,86811,2,0,1660497678,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI IDA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660502985,2022-08-14 18:50:08.921896
1,-43.247557,-22.812095,27204,2,49,1660500512,42A,GALEAO X MADUREIRA (PARADOR),ITI VOLTA - 42A GALEAO X MADUREIRA (PARADOR),1660502991,2022-08-14 18:50:08.921896
2,-43.313948,-22.853200,12707,2,32,1660501493,42A,GALEAO X MADUREIRA (PARADOR),ITI VOLTA - 42A GALEAO X MADUREIRA (PARADOR),1660502962,2022-08-14 18:50:08.921896
3,-43.345645,-22.888208,10306,4,36,1660485788,35,MADUREIRA X ALVORADA (PARADOR),ITI VOLTA - 35 MADUREIRA X ALVORADA (PARADOR),1660502978,2022-08-14 18:50:08.921896
4,-43.344860,-23.000292,13401,1,41,1660500457,22,TERMINAL ALVORADA X TERMINAL JARDIM OCEÂNICO (...,ITI VOLTA - 22 TERMINAL ALVORADA X TERMINAL J...,1660502996,2022-08-14 18:50:08.921896
5,-43.394555,-22.963892,30508,4,17,1660502728,50,TERMINAL OLÍMPICO X TERMINAL JARDIM OCEÂNICO (...,ITI IDA - 50 TERMINAL OLÍMPICO X TERMINAL JARD...,1660502979,2022-08-14 18:50:08.921896
6,-43.365567,-22.978728,12712,0,51,1660495774,46,PENHA X ALVORADA (EXPRESSO),ITI IDA - 46 PENHA X ALVORADA (EXPRESSO),1660502987,2022-08-14 18:50:08.921896
7,-43.339523,-23.000337,47015,6,59,1660501804,22,TERMINAL ALVORADA X TERMINAL JARDIM OCEÂNICO (...,ITI VOLTA - 22 TERMINAL ALVORADA X TERMINAL J...,1660503002,2022-08-14 18:50:08.921896
8,-43.562202,-23.004327,27201,6,56,1660501209,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI VOLTA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660502983,2022-08-14 18:50:08.921896
9,-43.684503,-22.917145,30527,4,0,1660485115,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI VOLTA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660502982,2022-08-14 18:50:08.921896


In [ ]:
data2.shape

(45, 11)

In [ ]:
for veiculo in data2:
  print(veiculo)
  #if veiculo.vei_nro_gestor in data.vei_nro_gestor.values:
  #  print(veiculo)

longitude
latitude
vei_nro_gestor
direcao
velocidade
inicio_viagem
linha
nomeLinha
nomeItinerario
comunicacao
coleta_atual


In [ ]:
veiculo in data2.vei_nro_gestor.values

True

In [ ]:
data2.head()

,longitude,latitude,vei_nro_gestor,direcao,velocidade,inicio_viagem,linha,nomeLinha,nomeItinerario,comunicacao,coleta_atual
0,-43.372515,-22.948763,44709,0,25,1660483651,35,MADUREIRA X ALVORADA (PARADOR),ITI IDA - 35 MADUREIRA X ALVORADA (PARADOR),1660488945,2022-08-14 14:55:50.749927
1,-43.323592,-22.855637,27174,1,0,1660487968,42A,GALEAO X MADUREIRA (PARADOR),ITI VOLTA - 42A GALEAO X MADUREIRA (PARADOR),1660488934,2022-08-14 14:55:50.749927
2,-43.365630,-22.985095,90005,0,53,1660488634,35,MADUREIRA X ALVORADA (PARADOR),ITI IDA - 35 MADUREIRA X ALVORADA (PARADOR),1660488919,2022-08-14 14:55:50.749927
3,-43.312110,-23.006988,30533,2,0,1660485921,50,TERMINAL OLÍMPICO X TERMINAL JARDIM OCEÂNICO (...,ITI IDA - 50 TERMINAL OLÍMPICO X TERMINAL JARD...,1660488946,2022-08-14 14:55:50.749927
4,-43.412097,-23.000133,47008,1,41,1660474598,10,SANTA CRUZ X ALVORADA (EXPRESSO),ITI IDA - 10 SANTA CRUZ X ALVORADA (EXPRESSO),1660488927,2022-08-14 14:55:50.749927


Fazer


- se veiculo não existe:

  então:
      add
  senão 

  se ele foi atualizado a mais de um minuto
  então: 
      atualiza
      

In [ ]:
data.coleta_atual

In [ ]:
print('atualizando os dados')

In [ ]:
'90002' in data.vei_nro_gestor.values 

True

In [ ]:
data.vei_nro_gestor.values

array(['30525', '30514', '13422', '12701', '90002', '13406', '12737',
       '44732', '30531', '13413', '90005', '47006', '13414', '47009',
       '30510', '27173', '13423', '27202', '12728', '30520', '30542',
       '86863', '30537', '47024', '44705', '12713', '27201', '86872',
       '30519', '47001', '86809', '47008', '47023', '44704', '47005',
       '30511', '12731', '12720', '13403', '12707', '12716', '12724',
       '12705', '13408', '13418', '12712', '47007', '47030', '47021',
       '30518'], dtype=object)

In [ ]:
!git clone --depth 1 https://github.com/PrefectHQ/prefect.git

Cloning into 'prefect'...
remote: Enumerating objects: 933, done.
remote: Counting objects: 100% (933/933), done.
remote: Compressing objects: 100% (809/809), done.
remote: Total 933 (delta 122), reused 460 (delta 77), pack-reused 0
Receiving objects: 100% (933/933), 9.27 MiB | 30.62 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [ ]:
%cd prefect/
!ls

/content/prefect
docker-compose.yml  mkdocs.insiders.yml  RELEASE-NOTES.md      setup.cfg
Dockerfile	    mkdocs.yml		 requirements-dev.txt  setup.py
docs		    netlify.toml	 requirements.txt      src
LICENSE		    orion-ui		 runtime.txt	       tests
MANIFEST.in	    README.md		 scripts	       versioneer.py


In [ ]:
import requests
import json
import pandas as pd
import datetime
import time

instante = datetime.datetime.now()
url_api = "http://citgisbrj.tacom.srv.br:9977/gtfs-realtime-exporter/findAll/json"
r = requests.get(url_api)

#json_request = json.loads(r.text)

#data = pd.DataFrame.from_dict(json_request, orient='columns')
#data['coleta_atual'] = instante

#data.head(5)

#response_list = []
